## Install YOLOv8

Using the latest model of Yolo which will have atleast more packages to handle the dataset and operations than other outdated versions of Yolo

Yolov8 is under heavy development.

In [3]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.13 torch-2.1.2+cpu CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5597.6/8062.4 GB disk)


In [4]:
from ultralytics import YOLO
from IPython.display import display, Image

## Preparing a custom dataset

Building a custom dataset is a painful process. It might take dozens or even hundreds of hours to collect images, label them, and export them in the proper format. But using Roboflow or makesense.ai, we can considerably atleast enable the task of labelling the dataset in such a way that yolo model which we will use will be comptible to work with the dataset.



### Step 1: Creating project

Activating Roboflow or makesense.ai to modify/customize the dataset systematicaly and successfully.

### Step 2: Uploading images
### Step 3: Generate new dataset version
### Step 4: Importing dataset

In [5]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow --quiet

from roboflow import Roboflow
rf = Roboflow(api_key="DdHFTNwdwoAnaDHza70t")
project = rf.workspace("zain-abideen").project("human-action-recognition-2-swqmx")
version = project.version(13)
dataset = version.download("yolov8")

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


/kaggle/working/datasets
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
jupyterlab 4.1.6 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
osmnx 1.9.2 requires shapely>=2.0, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you ha


Extracting Dataset Version Zip to Human-action-Recognition-2-13 in yolov8:: 100%|██████████| 1582/1582 [00:00<00:00, 6456.42it/s]


# The Modification Training

In [6]:
%cd {HOME}

!yolo task=segment mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=20 imgsz=640

/kaggle/working
100%|███████████████████████████████████████| 21.5M/21.5M [00:00<00:00, 236MB/s]
WARNING ⚠️ conflicting 'task=segment' passed with 'task=detect' model. Ignoring 'task=segment' and updating to 'task=detect' to match model.
New https://pypi.org/project/ultralytics/8.2.6 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.13 torch-2.1.2+cpu CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/working/datasets/Human-action-Recognition-2-13/data.yaml, epochs=20, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=F

In [7]:
!ls {HOME}/runs/detect/train3/

ls: cannot access '/kaggle/working/runs/detect/train3/': No such file or directory


In [8]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train3/confusion_matrix.png', width=600)

/kaggle/working


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/runs/detect/train3/confusion_matrix.png'

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train3/results.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train3/val_batch0_pred.jpg', width=600)

## Evaluate the Custom Model

In [ ]:
%cd {HOME}

!yolo task=segment mode=val model={HOME}/runs/detect/train3/weights/best.pt data={dataset.location}/data.yaml

## Relation with the modified Yolo Model

In [ ]:
%cd {HOME}
!yolo task=segment mode=predict model={HOME}/runs/detect/train3/weights/best.pt conf=0.25 source={dataset.location}/test/images save=true

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/detect/predict2/*.jpg')[:3]:
      display(Image(filename=image_path, height=600))
      print("\n")

## Saving the progress and deploying the model respectively.

Once you have finished training your YOLOv8 model, you’ll have a set of trained weights ready for use. These weights will be in the `/runs/detect/train/weights/best.pt` folder of your project. You can upload and your model weights to Roboflow Deploy for autolabeling, autoscaling inference, and using later.

The `.deploy()` function in the [Roboflow pip package](https://docs.roboflow.com/python) supports uploading YOLOv8 weights.

Run this cell to save your model weights:

In [ ]:
project.version(dataset.version).deploy(model_type="yolov8", model_path=f"{HOME}/runs/detect/train3/")

In [ ]:
# prompt: docker pull roboflow/roboflow-inference-server-gpu
#  /bin/bash: line 1: docker: command not found

!sudo apt-get install docker.io


In [ ]:
# prompt: deploy above model to streamlit

!pip install streamlit --quiet
!streamlit run {HOME}/runs/detect/train3/weights/best.pt
